<a href="https://colab.research.google.com/github/ErnestoAvedillo/AAM-math/blob/main/Reto_AAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargamos las librerias

In [54]:
#!pip install --upgrade gspread

In [55]:
#!pip install --upgrade pandas
#!pip install --upgrade Lib/datetime.py

In [56]:
import pandas as pd
import numpy as np
from google.colab import auth
from gspread.models import Cell
import gspread
from google.auth import default#autenticating to google
import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt

In [57]:
gspread.__version__

'3.4.2'

In [58]:
pd.__version__

'1.5.3'

#Reading Excel AAM data.gsheet


In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
file_path = "/content/drive/MyDrive/Colab Notebooks/AAM data.gsheet"
try:
    #https://docs.google.com/spreadsheets/d/179giTQNZghst9cpCtAzmTpHt0DrOtRQiBplU6-_XiUU/edit?usp=drive_link
    AAM_workbook = gc.open_by_key('179giTQNZghst9cpCtAzmTpHt0DrOtRQiBplU6-_XiUU')
    #worksheet = gc.open(file_path)
    print("Google Sheet opened successfully.")
except gspread.exceptions.SpreadsheetNotFound:
    print("Error: Spreadsheet not found.") #Convert to a DataFrame


Google Sheet opened successfully.


# **Primera parte**

Modelado de la producción

Cálculo del coste de las demandas internas.

known_references = nḿero de referencias (n).

REF = diccionario con las referencias de la empresa (TODAS , las de compra , internas  y de producto acabado.)

DC = Demanda de cliente

BOM = es una tabla de n x n con el BOM de los producto (Todos) Build of Materials

##Obteniendo un listado de las referencias.


In [61]:
BOM_worksheet = AAM_workbook.worksheet('BOM')#Convert to a DataFrame
rows = BOM_worksheet.get_all_values()#Convert to a DataFrame
df = pd.DataFrame(rows)
#creating columns name
df.columns = df.iloc[0]
df = df.iloc[1:]
REF_df = pd.concat([df['Ref Entrada'],df['Ref Salida']]).drop_duplicates()
#REF = REF_df[:].to_dict(orient='dict') does not work will try again
REF = {}
n = 0
for i in REF_df:
  REF[i] = n
  n += 1
print ("Diccionario de referencias con sus índices")
print(REF)

Diccionario de referencias con sus índices
{'AAA': 0, 'AMA': 1, 'BAJ': 2, 'AAB': 3, 'CAT': 4, 'AAE': 5, 'BAC': 6, 'CAR': 7, 'AAF': 8, 'BAD': 9, 'CAV': 10, 'AAC': 11, 'CAB': 12, 'CAX': 13, 'CAH': 14}


In [62]:
# Obtengo el numero de referencias
known_references = REF_df.shape[0]
print("Número total de referencias ", known_references)

Número total de referencias  15


## Creando el BOM que aparece en el Excel

In [63]:
#BILL OF MATERIALS
BOM = np.zeros((known_references,known_references))
for index, row in df.iterrows():
  BOM[REF[row["Ref Salida"]],REF[row["Ref Entrada"]]] = float(row["Cantidad"].replace(',', '.'))
BOM_df = pd.DataFrame(BOM,index=REF_df, columns=REF_df)
print (BOM_df)

     AAA   AMA  BAJ  AAB  CAT  AAE  BAC  CAR  AAF  BAD  CAV  AAC  CAB  CAX  \
AAA  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AMA  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAJ  1.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAB  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAT  0.0  0.07  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAE  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAC  0.0  0.00  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAR  0.0  0.08  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAF  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAD  0.0  0.00  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAV  0.0  0.05  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
AAC  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [64]:
BOM = BOM_df.values
mask_compras = np.expand_dims(BOM.sum(axis=1) == 0, axis = 0)
mask_cliente = np.expand_dims(BOM.sum(axis=0) == 0, axis = 0)

## Obtención de la demanda de cliente

In [65]:
DC_worksheet = AAM_workbook.worksheet('DC') #Convert to a DataFrame
rows = DC_worksheet.get_all_values()#Convert to a DataFrame
DC_df = pd.DataFrame(rows)
DC_df.iloc[0][0] = None
# Set row 1 as column names
DC_df.columns = DC_df.iloc[0]

# Set column 1 as row names
DC_df = DC_df.set_index(DC_df.iloc[:, 0])

# Remove row 1 and column 1
DC_df = DC_df.iloc[1:, 1:]
print(DC_df)
#Demandas de cliente
Profundidad = 15
DC = np.zeros((known_references,Profundidad))
for index, row in DC_df.iterrows():
  DC[REF[index],0] = row["Total"]
print('\n')
print (DC)

0   Total Piezas/caja
CAB   560            
CAX   300            
CAH   320            


[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0

##Descargamos los datos de almacen las referencias



In [66]:
Almacen_worksheet = AAM_workbook.worksheet('ALMACEN') #Convert to a DataFrame
rows = Almacen_worksheet.get_all_values()#Convert to a DataFrame
Almacen_df = pd.DataFrame(rows)
Almacen_df.iloc[0][0] = None
# Set row 1 as column names
Almacen_df.columns = Almacen_df.iloc[0]

# Set column 1 as row names
Almacen_df = Almacen_df.set_index(Almacen_df.iloc[:, 0])

# Remove row 1 and column 1
Almacen_df = Almacen_df.iloc[1:, 1:]
print(Almacen_df)
#Demandas de cliente

0   STOCK   Piezas caja €/UD Stock minimo Stock maximo Stock stop Lote max  \
AAA  2000            16   15          240          240        240     1000   
AAB  2000            20   23          240          240        240     1000   
AAC  2000            18   50          240          240        240     1000   
AAE  2000            18   44          240          240        240     1000   
AAF  2000            16   15          240          240        240     1000   
AMA  2000            20   98          240          240        240     1000   
CAB    32            32   71          240          240        240      704   
CAH    36            36   30          240          240        240      720   
CAX    20            20   33          240          240        240      720   
BAC    40            18   44           88          220        352      720   
BAD     0            32   15           30           75        120      704   
BAJ    48            16   98          196          490        78

In [67]:
Almacen = np.zeros((known_references, 9))
for index, row in Almacen_df.iterrows():
  Almacen[REF[index],0] = row["STOCK"]
  Almacen[REF[index],1] = row["Piezas caja"]
  Almacen[REF[index],2] = row["€/UD"]
  Almacen[REF[index],3] = row["Stock minimo"]
  Almacen[REF[index],4] = row["Stock maximo"]
  Almacen[REF[index],5] = row["Stock stop"]
  Almacen[REF[index],6] = row["Lote max"]
  Almacen[REF[index],7] = row["Lote min"]
  Almacen[REF[index],8] = row["T.CICLO"]
print(Almacen)

[[2000.   16.   15.  240.  240.  240. 1000.    0.    0.]
 [2000.   20.   98.  240.  240.  240. 1000.    0.    0.]
 [  48.   16.   98.  196.  490.  784.  720.    0.   55.]
 [2000.   20.   23.  240.  240.  240. 1000.    0.    0.]
 [  32.   20.   30.   60.  150.  240.  720.    0.   30.]
 [2000.   18.   44.  240.  240.  240. 1000.    0.    0.]
 [  40.   18.   44.   88.  220.  352.  720.    0.   50.]
 [   0.   16.   71.  142.  355.  568.  720.    0.   40.]
 [2000.   16.   15.  240.  240.  240. 1000.    0.    0.]
 [   0.   32.   15.   30.   75.  120.  704.    0.   60.]
 [  36.   60.   33.   66.  165.  264.  720.    0.   40.]
 [2000.   18.   50.  240.  240.  240. 1000.    0.    0.]
 [  32.   32.   71.  240.  240.  240.  704.  200.  120.]
 [  20.   20.   33.  240.  240.  240.  720.  400.  120.]
 [  36.   36.   30.  240.  240.  240.  720.  300.  120.]]


Extraemos la informacion de las siguientes matrices:

Stock: Strock inicial

BOX: Piezs por caja

Prices: Coste de carda referencia

Matrix_Decision: matriz para la toma de decision de qué es lo que producimos

T: Tiempos de ciclo

In [68]:
#extraigo el stock inicial
Stock_ini = np.expand_dims(np.copy(Almacen[:,0]), axis = 0)
#Extraigo las piezas por caja
BOX = np.expand_dims(np.copy(Almacen[:,1]), axis = 0)
#extraigo los precios de cada referencia
Prices = np.expand_dims(np.copy(Almacen[:,2]), axis = 0)
#Extraigo la matriz de decisiones
Matrix_decision = np.copy(Almacen[:,3:5])
#Extraigo los valores del lote_m,inimo y máximo
Matrix_lote_min = np.expand_dims(np.copy(Almacen[:,7]),axis = 0)
Matrix_lote_max = np.expand_dims(np.copy(Almacen[:,6]),axis = 0)
#Extraigo los tiempos de ciclo
T = np.expand_dims(np.copy(Almacen[:,8]), axis = 0)
T_current = np.expand_dims(np.copy(Almacen[:,8]), axis = 0)
T_current [mask_compras] = -1


##Procedimiento de cálculo de las demandas internas

In [69]:
#Voy incremetnando la matriz de demandas a medida que aumenta el nicel de fabricación
def Calcula_necesidades_por_nivel(DC,BOM_entrada,Profundidad):
  for profIndex in range(Profundidad-1):
    DC[:,profIndex+1] = np.dot(BOM_entrada.T,DC[:,profIndex])
  return(DC)

In [70]:
#Voy incremetnando la matriz de demandas a medida que aumenta el nicel de fabricación
def Calcula_necesidades(DC, BOM_entrada, Profundidad):
  DC = Calcula_necesidades_por_nivel(BOM_entrada,Profundidad)
  return(DC.sum(axis = 1))

##Tabla de cálculo de stocks mínimos
Esta table esvirá para conocer si disponemos de stock suficiente para trabajar en el proceso considerando que queremos realizar el stock mínimo.


In [71]:
Factor_BOM = np.copy(BOM)
Factor_BOM[Factor_BOM > 0] = 1
Factor_BOM = np.divide(np.multiply(Factor_BOM, T).T,T).T
#Elimino los NAN y los INF
Factor_BOM = np.where(np.isnan(Factor_BOM),0,Factor_BOM)
Factor_BOM = np.where(np.isinf(Factor_BOM),0,Factor_BOM)
Demandas_cliente = np.zeros((known_references,Profundidad))
Demandas_cliente [:,0] = Matrix_lote_min
Sumando1 = Calcula_necesidades_por_nivel(Demandas_cliente, BOM, Profundidad).astype(int)
Sumando2 = Calcula_necesidades_por_nivel(Demandas_cliente, Factor_BOM, Profundidad).astype(int)
Matrix_lote_min_internos = Sumando1 - Sumando2
df4 = pd.DataFrame(Matrix_lote_min_internos, index=REF.keys(), columns= np.arange(0,15))
print(df4)
Stock_minimo = Matrix_lote_min_internos.sum(axis = 1)
print(Stock_minimo)


     0    1    2    3   4   5   6   7   8   9   10  11  12  13  14
AAA   0    0    0  400   0   0   0   0   0   0   0   0   0   0   0
AMA   0    0   75    0   0   0   0   0   0   0   0   0   0   0   0
BAJ   0    0  309    0   0   0   0   0   0   0   0   0   0   0   0
AAB   0  420    0    0   0   0   0   0   0   0   0   0   0   0   0
CAT   0  350    0    0   0   0   0   0   0   0   0   0   0   0   0
AAE   0    0    0  700   0   0   0   0   0   0   0   0   0   0   0
BAC   0    0  234    0   0   0   0   0   0   0   0   0   0   0   0
CAR   0  267    0    0   0   0   0   0   0   0   0   0   0   0   0
AAF   0  400    0    0   0   0   0   0   0   0   0   0   0   0   0
BAD   0    0  150    0   0   0   0   0   0   0   0   0   0   0   0
CAV   0  200    0    0   0   0   0   0   0   0   0   0   0   0   0
AAC   0  270    0    0   0   0   0   0   0   0   0   0   0   0   0
CAB   0    0    0    0   0   0   0   0   0   0   0   0   0   0   0
CAX   0    0    0    0   0   0   0   0   0   0   0   0   0   0

<ipython-input-71-c8cc2eb0b252>:3: RuntimeWarning: invalid value encountered in true_divide
  Factor_BOM = np.divide(np.multiply(Factor_BOM, T).T,T).T


Simulacion de un BOM con dos productos diferentes

In [115]:
def Actualiza_stock(mask_finished_part, Stock_ini):
  produced = np.zeros((known_references,1))
  produced[mask_finished_part.T] = 1
  Stock_ini = Stock_ini + produced.T
  Stock_ini = Stock_ini - np.dot(BOM.T, produced).T
  T_current[mask_finished_part] = T[mask_finished_part]
  return(Stock_ini)

In [173]:
#Funcion para el cálculo de piezas producidas en un tiempo determinado en segundos.
def Produce( seconds, T_current,  Stock_ini, T):
  i = seconds
  while i > 0 :
    mask_only_positives = T_current > 0
    Value = np.min(T_current[T_current > 0])
    i = i - min (i , Value)
    T_current = T_current - np.ones(known_references) * min (i , Value)
    mask_finished_part = T_current == np.zeros((known_references))
    if (mask_finished_part.any() == True):
      Stock_ini = Actualiza_stock(mask_finished_part, Stock_ini)
      T_current[mask_finished_part] = T[mask_finished_part]
    is_negative = np.any(Stock_ini <= 0)
    if is_negative:
      break
  return(T_current, Stock_ini)


In [174]:
def display_bar_graph (ax, Labels, Values):
  ax.bar(Labels, Almacen)
  plt.xlabel('References')
  plt.ylabel('Stock')
  plt.title('Stock for each reference')
  # Display the initial bar graph
  plt.show()
  return(plt)

In [188]:
from IPython.display import clear_output
import plotly.express as px
import plotly
# Set the matplotlib backend
%matplotlib inline
# Create the bar graph
#Almacen = Stock_minimo + DC [:,0]
Almacen = Stock_minimo + DC [: , 0]
Almacen_puntual_df = pd.DataFrame(np.expand_dims(Almacen, axis = 0), columns = REF.keys())
#pongo un tiepo de visualizacion en minutos
T_current = T
T_current [T == 0] = -1
Ciclos_visualizacion = dias * 24 * 3600 / Tiempo_visualizacion
Ciclos_visualizacion = int(Ciclos_visualizacion)
print(Ciclos_visualizacion)
for i in range (1, Ciclos_visualizacion):
  T_current,  Almacen = Produce( 900, T_current,  Almacen, T)
  mask_finished_part = T_current == 0
  Almacen = Actualiza_stock(mask_finished_part, Almacen)
  #Update the bar heights
  current_df = pd.DataFrame(Almacen, columns = REF.keys())
  Almacen_puntual_df = pd.concat([Almacen_puntual_df, current_df])
  is_negative = np.any(Almacen <= 0)
  if is_negative:
    break
print(Almacen_puntual_df)

17280
     AAA    AMA    BAJ    AAB    CAT    AAE    BAC    CAR    AAF    BAD  \
0  400.0  75.00  309.0  420.0  350.0  700.0  234.0  267.0  400.0  150.0   
0  384.0  70.11  296.0  410.9  365.0  669.0  229.0  282.0  393.0  142.0   
0  368.0  65.15  282.0  401.8  381.0  636.0  225.0  297.0  386.0  135.0   
0  352.0  60.26  269.0  391.4  394.0  603.0  221.0  311.0  378.0  128.0   
0  336.0  55.37  256.0  382.3  409.0  572.0  216.0  326.0  371.0  120.0   
0  320.0  50.41  242.0  373.2  425.0  539.0  212.0  341.0  364.0  113.0   
0  304.0  45.52  229.0  362.8  438.0  507.0  207.0  355.0  356.0  106.0   
0  288.0  40.63  216.0  353.7  453.0  475.0  203.0  370.0  349.0   98.0   
0  272.0  35.67  202.0  344.6  469.0  442.0  199.0  385.0  342.0   91.0   
0  256.0  30.78  189.0  334.2  482.0  410.0  194.0  399.0  334.0   84.0   
0  240.0  25.89  176.0  325.1  497.0  378.0  190.0  414.0  327.0   76.0   
0  224.0  20.93  162.0  316.0  513.0  346.0  185.0  429.0  320.0   69.0   
0  208.0  16.04  14

In [181]:
!pip install bar_chart_race

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 4.1 MB/s eta 0:00:00


In [189]:
#printing results
import bar_chart_race as bcr
import plotly
columns = REF.keys()
data = Almacen_puntual_df[columns]
fig = px.bar(Almacen)
bcr.bar_chart_race(df = Almacen_puntual_df,filename = None, figsize = (3.5 , 3), title =" Stock por referencia")

/usr/local/lib/python3.10/dist-packages/bar_chart_race/_make_chart.py:286: UserWarning:

FixedFormatter should only be used together with FixedLocator

/usr/local/lib/python3.10/dist-packages/bar_chart_race/_make_chart.py:287: UserWarning:

FixedFormatter should only be used together with FixedLocator

/usr/local/lib/python3.10/dist-packages/bar_chart_race/_make_chart.py:428: UserWarning:

This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.



**Listado de Maquinaria**

mumMaquinas = número de máquinas

DICMAQ = Diccionario de maquinaria (m x 1)

BOMMAQ = Listado de maquinaria por referencia (m x n)

##Descargamos los datos de las máquinas

In [ ]:
Maquinas_worksheet = AAM_workbook.worksheet('MAQUINARIA') #Convert to a DataFrame
rows = Maquinas_worksheet.get_all_values()#Convert to a DataFrame
MAQUINAS_df = pd.DataFrame(rows)
# Set row 1 as column names
MAQUINAS_df.columns = MAQUINAS_df.iloc[0]
# Remove row 1 and column 1
MAQUINAS_df = MAQUINAS_df.iloc[1:,:]
print(MAQUINAS_df)


##Creamos el diccionario de las máquinas

In [ ]:
DICMAQ = {"Compras":0}
MAQ_df = MAQUINAS_df['COD. MAQUINA'].drop_duplicates().sort_values()
print ("dataframe de Maquinas")
print(MAQ_df)
n = 1
for i in MAQ_df:
  DICMAQ[i] = n
  n += 1
print('\n')
print("Diccionario de máquinas")
print(DICMAQ)

numMaquinas = MAQ_df.shape[0] + 1
print ("Número de máquinas" , numMaquinas)



## Creamos el BOM de la maquinaria (Referencia vs Maquina)

In [ ]:
Procesos_worksheet = AAM_workbook.worksheet('PROCESOS') #Convert to a DataFrame
rows = Procesos_worksheet.get_all_values()#Convert to a DataFrame
Procesos_df = pd.DataFrame(rows)
# Set row 1 as column names
Procesos_df.columns = Procesos_df.iloc[0]
# Remove row 1 and column 1
Procesos_df = Procesos_df.iloc[1:,:]
print(Procesos_df)

In [ ]:
BOMMAQ = np.zeros((numMaquinas,known_references))
for index, row in Procesos_df.iterrows():
  BOMMAQ[DICMAQ[row["PROCESO"]], REF[row["Ref Salida"]]] = 1
print (BOMMAQ)

In [ ]:
DC_internas = Calcula_necesidades(DC,BOM,Profundidad)
print("Demandas para el día", '\n')
print (DC_internas)

In [ ]:
Matrix_necessity = DC_internas - Stock_ini
Matrix_necessity[mask_compras] = 0
Matrix_necessity_per_maq = np.multiply(Matrix_necessity,BOMMAQ)
mask_Kanban_red = Matrix_necessity_per_maq >= Matrix_decision[:,0]
print(Matrix_necessity_per_maq)

In [ ]:
#Funcion de toma de decisión de qué maquina ha de trabajar
def Making_decision(DC_internas):
  Matrix_necessity = DC_internas - Stock_ini
  Matrix_necessity[mask_compras] = 0
  Matrix_necessity_per_maq = np.multiply(Matrix_necessity,BOMMAQ)
  for i in range(0, numMaquinas):
    mask_Kanban = Matrix_necessity[i] <= Matrix_decision[:,0]
    if(not np.all(mask_Kanban[i])):
      mask_Kanban = Matrix_necessity[i] <= Matrix_decision[:,1]
    val_min= np.min(Matrix_necessity[i,mask_Kanban])
    mask_Kanban[i] = np.logic_and(mask_Kanban,Matrix_necessity[i,mask_Kanban] == val_min)
  print(mask_Kanban)
  for i in range(0,numMaquinas):
    mask_maquina = BOMMAQ[i,:] == 1

  aux_array = np.multiply(np.ones(Matrix_necessity.shape).T, Stock_ini)
  mask_necesity = Matrix_decision > aux_array.T
  Production_array = np.copy(Stock_ini)
  Production_array[0,np.logical_not(mask_necesity[:,0])] = 0
  return(Production_array)

In [ ]:
Vector_produccion = Making_decision(DC_internas)
print (Vector_produccion)


In [ ]:
Stock_ini = np.expand_dims(np.copy(Almacen[:,0]), axis = 0)
T_current = np.copy(T)
print (T_current)
print(Stock_ini)
T_current, Stock_ini = Produce(35, T_current, Stock_ini)
print ("salida")
print(Stock_ini)
print (T_current)


In [ ]:
#Busco en número de cajas necesario para la produccion
DCBOX = np.copy(DC);
for dayIndex in range(Dias):
  mask = np.mod(DC[:,:,dayIndex].T, BOX) > 0
  ones_array = np.zeros_like(np.mod(DC[:,:,dayIndex], BOX.T))
  ones_array [mask.T] = 1
  DCBOX[:,:,dayIndex] = np.floor_divide(DC[:,:,dayIndex], BOX.T) +  ones_array
  print (DCBOX[:,:,dayIndex],'\n')



MAQ = is a vector of ONES where there is an operation


In [ ]:
MAQ = np.zeros((numMaquinas,Dias));
for dayIndex in range(Dias):
  MAQ[:,dayIndex] = np.dot(DCHORAS[:,:,dayIndex].sum(axis = 1), BOMMAQ.T)
  print(MAQ[:,dayIndex],'\n')

In [ ]:
# Calculo el numero de cajas para cada referencia
Cajas_stock_global = np.divide(Stock_ini , BOX)
print (Cajas_stock_global)

##Separo stock por almacenes

In [ ]:
Sim_almacen = np.ones((known_references,3))
mask1 = np.logical_not (np.sum(BOM, axis = 1) == 0)
mask3 = np.logical_not (np.sum(BOM, axis = 0) == 0)
mask2 = np.logical_not (mask1 & mask3)
#Sim_almacen = np.multiply(Sim_almacen, Cajas_stock_global)
Sim_almacen [mask1,0] = 0
Sim_almacen [mask2,1] = 0
Sim_almacen [mask3,2] = 0
print (Sim_almacen)

In [ ]:
#Comparo el stock de cajas con la capacidad del almacen
Capa_almac_worksheet = AAM_workbook.worksheet('CAPACIDAD ALMACEN') #Convert to a DataFrame
rows = Capa_almac_worksheet.get_all_values()#Convert to a DataFrame
Capa_almac_df = pd.DataFrame(rows)
# Set row 1 as column names
Capa_almac_df.columns = Capa_almac_df.iloc[0]
# Remove row 1 and column 1
Capa_almac_df = Capa_almac_df.iloc[1:,:]
print(Capa_almac_df)

In [ ]:
Capa_alm = np.zeros((3,1))
for index, row in Capa_almac_df.iterrows():
  Capa_alm[index - 1] = row["CANTIDAD"]
print (Capa_alm)
print(np.sum(Sim_almacen,axis = 0))
Almacen_capaz = np.all(np.sum(Sim_almacen,axis = 0) < Capa_alm)
print(" El almacen es capaz", Almacen_capaz)

# Definición de Bases de la Simulación
## Constraints de la simulacion

LoteMin = Lote mínimo de producción (n x 1)

LotMax = Lote máximo de producción (n x 1)

TCambio = Tiempos de cambio de referencia (n x n)

Planif =  Planificacion de la producción (Maq x 24 horas x Ref)

PrioMat = Matriz de priodidades (n x 1)

*Donde n = known_references*

In [ ]:
Transiciones_worksheet = AAM_workbook.worksheet('TRANSICIONES') #Convert to a DataFrame
rows = Transiciones_worksheet.get_all_values()#Convert to a DataFrame
Transiciones_df = pd.DataFrame(rows)
# Set row 1 as column names
Transiciones_df.columns = Transiciones_df.iloc[0]
# Remove row 1 and column 1
Transiciones_df = Transiciones_df.iloc[1:,:]
print(Transiciones_df)


In [ ]:
Transiciones = np.zeros((numMaquinas,known_references, known_references))
for index, row in Transiciones_df.iterrows():
  Transiciones[DICMAQ[row["PROCESO"]], REF[row["Origen"]], REF[row["Destino"]]] =  row["Tiempo"]
for key in DICMAQ.keys():
  print ("Tiempos de transición para la máquina ", key, '\n')
  print (Transiciones[DICMAQ[key],:,:])

In [ ]:
print (MAQ)
print (DCHORAS[:,:,0])

# Cálculo de costes del proceso

En esta sección calcularemos los costes del proceso.

En esencia el cálculo es el siguiente:

Coste proceso [CP] = Coste mano de obra[CMO] + Coste Máquina [CMQ]

Desarrollando un poco mas este cálculo tenemos que:

 CMO = Coste operario [MO] / Tiempo de ciclo [T] / 3600

CMQ = Coste Máquina fijo [CMQF] Coste Máquina Variable  [CMQV]

CMQF = Coste fijo Máquina [CF] * T / 3600

CMQV = Coste potencia [CPOT] * Potencia Máquina [POT] * T / 3600

Sustituyento tenemos:

CP = (MO + CF + CPOT * POT) * T / 3600

Ahora nos centraremso en el cálculo de la mano de obra

MAQREF = Vector Maquinas (n x 1)

MOM = Coste de la mano de obra Turno de mañana

MOT = Coste de la mano de obra Turno de tarde

MON = Coste de la mano de obra Turno de noche

MHORARIA = MATRIZ Horaria (n x 24)

NOP =  Número de operarios (n x 1) (Dato a recibir de cliente)

CMOTHORARIO = Coste de mano de obra total (nx24) resultado de MOTURNOS * MOP *T / 3600

In [ ]:
MAQREF = np.zeros((known_references,1))
mask = BOMMAQ.sum(axis = 0) > 0
MAQREF[mask] = 1
print(MAQREF)
MHORARIA = np.ones((known_references,24))
MOTURNOS = np.multiply(MHORARIA, MAQREF)
print(MOTURNOS)

In [ ]:
MOM = 12
MOT = 14
MON = 18
MOTURNOS [:,0:6] *=  MON
MOTURNOS [:,6:14] *=  MOM
MOTURNOS [:,14:22] *=  MOT
MOTURNOS [:,22:24] *=  MON
print(MOTURNOS)

In [ ]:
CMOTHORARIO  =np.multiply(MOTURNOS, np.multiply(T,NOP)) / 3600
print(CMOTHORARIO)


Ahora nos centraremso en el cálculo de la mano de obra

MAQ = Vector Maquinas (n x 1)

CMF = Coste máquina fijo (n x 24)

CPOT = Tabla de coste de potencias (n x 24)

POT = Potencia de Maquinaria (n x 1) KWH

CELECM = Coste potencia turno mañana

CELECT = coste potencia turno tarde

CELECN = Coste potencia turno Noche

CMVTURNOS = Coste maquina variable (n x 24))

T =  Tiempos de ciclo (n x 1) (Definidos anteriormente)

CMQVTHORARIO = Coste de maquina total (nx24) resultado de CMVTURNOS * POT *T / 3600

CMTOT = (CMF + CMQVTHORARIO) * T / 3600

In [ ]:
from genericpath import commonprefix
#Coste maquina definido por cliente CMF
cmfijo = 5  #5 es un valor estimado
CMF = np.ones((known_references,24))
CMF = np.multiply(CMF, MAQREF)
CMF = CMF * cmfijo
print (CMF)

In [ ]:
CMTURNOS = np.ones((known_references,24))
CMTURNOS = np.multiply(CMTURNOS, MAQREF)
print(CMTURNOS)

In [ ]:

CELECM = 0.12
CELECT = 0.14
CELECN = 0.16
CMTURNOS [:,0:6] *=  CELECN
CMTURNOS [:,6:14] *=  CELECM
CMTURNOS [:,14:22] *=  CELECT
CMTURNOS [:,22:24] *=  CELECN
print(CMTURNOS)

In [ ]:
POT_referencia = np.multiply(BOMMAQ,CPOT)
POT_referencia = np.expand_dims(POT_referencia.sum(axis = 0), axis = 0)

print(POT_referencia)

In [ ]:
CMQVTHORARIO  = np.multiply(CMTURNOS , POT_referencia.T)
print(CMQVTHORARIO)

In [ ]:
CMTOT = np.multiply((CMF + CMQVTHORARIO), T) / 3600
print (CMTOT)

El coste total es el resultado de sumar el coste máquina y el de operario
CTOTHORARIO

In [ ]:
CTOTHORARIO = CMOTHORARIO + CMTOT
print (CTOTHORARIO)

# **Inicio de modelado de simulación**
El modelo de la simulación requiere que se definan:
1. Descipción de los componentes del sistema (ver zona anterior)
2. El reloj de la simulación (arreglo de tiempos) se considera trabajar por semanas de 6 días 0-lunes -- 5 sábado
3. Definición del estado inicial del sistema
4. El modelo para el cambio de estado
5. Procedimiento para adelantar el reloj
Como resultado del sistema se provee:
1. Condición inicial
2. Listado de eventos (aleatorios + paramétricos)
3. Resultado obtenido de la ejecución en parámetros evaluables de la efectividad del resultado.

*Reloj de la simulación* - se establece como una matriz horarira de 24 x 6 días (programación semanal)

In [ ]:
Calendario_Sim = np.zeros((24,6))
print(Calendario_Sim)

*Definir el estado inicial* - Estado de las máquinas y almacenamiento

Se definie un Coste de referencia para el calculo del coste de material en el almacén

In [ ]:
Process_status = np.zeros((len(DICMAQ),1))
Process_status[DICMAQ["Inyeccion"]] = REF["5"]
Process_status[DICMAQ["Mecanizado"]] = REF["8"]
STOCK = {"Referencia":0,"Volumen":1,"Coste":2}
Reference_cost = np.zeros((len(REF),1))
Reference_cost[REF["1"]] = 12.8
Reference_cost[REF["2"]] = 23.7
Reference_cost[REF["3"]] = 18.4
Reference_cost[REF["4"]] = 6.2
Reference_cost[REF["5"]] = 50.1
Reference_cost[REF["6"]] = 80.2
Reference_cost[REF["7"]] = 67.4
Reference_cost[REF["8"]] = 73.9

Almacen_MP = np.zeros((3,known_references))
Almacen_MP[STOCK["Referencia"],REF["1"]] = REF["1"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["1"]] = 5
Almacen_MP[STOCK["Coste"],REF["1"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["1"]]

Almacen_MP[STOCK["Referencia"],REF["2"]] = REF["2"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["2"]] = 7
Almacen_MP[STOCK["Coste"],REF["2"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["2"]]

Almacen_MP[STOCK["Referencia"],REF["3"]] = REF["3"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["3"]] = 1
Almacen_MP[STOCK["Coste"],REF["3"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["3"]]

Almacen_MP[STOCK["Referencia"],REF["4"]] = REF["4"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["4"]] = 2
Almacen_MP[STOCK["Coste"],REF["4"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["4"]]


# Notas Alf - Borrar
#MAQ_TRANS_TIEMPO[2,TRANS["Maquina"]] = DICMAQ["Inyeccion"] / "Mecanizado"
#MAQ_TRANS_TIEMPO[2,TRANS["Origen"]] = REF["5"]             / REF["7"]
#MAQ_TRANS_TIEMPO[2,TRANS["Destino"]] = REF["6"]            / REF["8"]

print("Reference costs:",Reference_cost,"\n")
print("Current status of RowMat Stock\n",Almacen_MP)
print("Process Current State: ",Process_status)

In [ ]:
from pandas.compat import numpy
# Production diary - this is an array where
# 0. Timeframe recorded
# 1. Stock status
sim_start = dt.datetime(year=2023, month=6, day=23, hour=0, minute=0, second=0)
step = timedelta(minutes=15)
sim_frame = sim_start
sim_log = []
print(Stock_ini)
for i in range(0,12):
  log_register = []
  print("Frame [",i,"] ",sim_frame)
  sim_frame += step
  log_register.append(sim_frame.strftime("%Y-%m-%d %H:%M:%S"))
  log_register.append(Stock_ini)
  #sim_log.append(sim_frame.strftime("%Y-%m-%d %H:%M:%S"))
  sim_log.append(log_register)

sheet_name = 'ProductionDiary'
sheet.clear()
sheet = AAM_workbook.worksheet(title=sheet_name)
cells = []
for i, log_entry in enumerate(sim_log):
  cells.append(Cell(row=i+2, col=1, value=log_entry[0]))
  for j, stock_entry in enumerate(np.array(log_entry[1])):
    for k, stock_entry2 in enumerate(stock_entry):
      cells.append(Cell(row=i+2, col=k+2, value=stock_entry[k]))

sheet.update_cells(cells)
#print("SIM Log:\n",sim_log)


In [ ]:
Graficar resultados